In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

# Load CIFAR-10 dataset from Keras
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize images
x_train, x_test = x_train.astype('float32') / 255.0, x_test.astype('float32') / 255.0
y_train, y_test = tf.keras.utils.to_categorical(y_train, 10), tf.keras.utils.to_categorical(y_test, 10)

# Combine train and test data to create a larger subset, if desired
x_data = np.concatenate((x_train, x_test))[:30000]
y_data = np.concatenate((y_train, y_test))[:30000]

# Split into training and validation sets
split_index = int(0.8 * len(x_data))
x_train, x_val = x_data[:split_index], x_data[split_index:]
y_train, y_val = y_data[:split_index], y_data[split_index:]

# Load VGG16 without top layers and freeze layers for transfer learning
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of VGG16
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dense(10, activation='softmax')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with 3 epochs
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=3,
    batch_size=64
)

# Evaluate on validation data
val_loss, val_accuracy = model.evaluate(x_val, y_val)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

# Function to display a few images with predictions
def display_predictions(model, x_data, y_data, num_images=5):
    plt.figure(figsize=(15, 5))
    for i in range(num_images):
        # Select a random image from the validation set
        index = np.random.randint(0, x_data.shape[0])
        image = x_data[index]
        true_label = np.argmax(y_data[index])

        # Get the model's prediction
        prediction = model.predict(image[np.newaxis, ...])
        predicted_label = np.argmax(prediction)

        # Plot the image
        plt.subplot(1, num_images, i + 1)
        plt.imshow(image)
        plt.title(f'True: {true_label}, Pred: {predicted_label}')
        plt.axis('off')
    plt.show()

# Display predictions for a few images
display_predictions(model, x_val, y_val, num_images=5)
